In [1]:
import json

rxnfile=open("CodeSystem-rxnorm-03072022.json")
rxn = json.load(rxnfile)

import csv

# snomeds = []
# # open the CSV file in read mode
# with open('SNOMEDCT_CORE_SUBSET_202211/SNOMEDCT_CORE_SUBSET_202211.txt', 'r') as csvfile:
#     # create a CSV reader object
#     csvreader = csv.reader(csvfile, delimiter="|")
    
#     # get the header row and create a dictionary of column indexes
#     header = next(csvreader)
#     indexes = {header[i]: i for i in range(len(header))}

#     # loop over the remaining rows and create a dictionary for each row
#     for row in csvreader:
#         snomed = {}
#         for key, index in indexes.items():
#             snomed[key] = row[index]
#         snomed["text"] = snomed["SNOMED_FSN"]
#         snomeds.append(snomed)

# len(snomeds)

In [2]:
def rxntype(c):
    try:
        return list(filter(lambda v: v in ['SBD', 'SCD', 'IN', 'BN', 'BPCK', 'GPCK'], [p['valueString'] for p in c['property'] if p['code']=='tty']))[0]
    except:
        return None

rxns = []
for c in rxn['concepts']:
    tty = rxntype(c)
    if tty:
        rxns.append({'code': c['code'],  'display': c['display'], 'text': c['display'], 'tty': tty})
len(rxns)

import nltk
nltk.download("punkt")
token_counts = {}
for r in rxns:
    r["tokens"] = set([v.lower() for v in nltk.tokenize.word_tokenize(r['display'])])
    for t in r["tokens"]:
        if t not in token_counts:
            token_counts[t] = 0
        token_counts[t] += 1
len(token_counts)


[nltk_data] Downloading package punkt to /home/jmandel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


12032

In [5]:
import sqlite3
conn = sqlite3.connect("rxnorm.db")

# Create a cursor object
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS vocab")
cur.execute("CREATE VIRTUAL TABLE IF NOT EXISTS vocab USING fts5(code, display, related, common_test_rank, units, tokenize = porter)")
cur.execute("INSERT into vocab(vocab, rank) values('rank', ' bm25(100.0, 10, 5, 0)')")

for r in rxns:
    code = r['code']
    display = r['display']
    related = ""
    common_test_rank = 0
    units = ""
    cur.execute("INSERT INTO vocab (code, display, related, common_test_rank, units) VALUES (?, ?, ?, ?, ?)", (code, display, related, common_test_rank, units));

# Define a function to perform a full-text search on the table
def search_loinc(query, limit = 100, skip = 0):
  q = " OR ".join(query.replace("-", " ").split(" "))
  print(q)
  cur.execute("select * from (SELECT *, rank FROM vocab WHERE vocab MATCH ? ORDER BY rank limit ? offset ?) order by common_test_rank", (q, limit, skip,))
  results = cur.fetchall()
  print(f"---\nSearch results for '{query}':")
  for row in results:
    print(f"{row[0]}, {row[1]} ctr {row[3]} {row[-1]}")

# Perform some sample searches
search_loinc("acetaminophen 325")
conn.commit()
conn.close()

acetaminophen OR 325
---
Search results for 'acetaminophen 325':
198436, acetaminophen 325 MG Oral Capsule ctr 0 -18.869460967461148
198437, acetaminophen 325 MG Rectal Suppository ctr 0 -18.869460967461148
2264779, acetaminophen 325 MG Chewable Tablet ctr 0 -18.869460967461148
313782, acetaminophen 325 MG Oral Tablet ctr 0 -18.869460967461148
603136, acetaminophen 325 MG Disintegrating Oral Tablet ctr 0 -18.683534117645873
1049214, oxyCODONE 10 MG / acetaminophen 325 MG Oral Tablet ctr 0 -18.322459970477404
1049221, oxyCODONE 5 MG / acetaminophen 325 MG Oral Tablet ctr 0 -18.322459970477404
1724446, butalbital 25 MG / acetaminophen 325 MG Oral Tablet ctr 0 -18.322459970477404
197426, butalbital 50 MG / acetaminophen 325 MG Oral Tablet ctr 0 -18.322459970477404
247816, acetaminophen 325 MG / caffeine 65 MG Oral Tablet ctr 0 -18.322459970477404
313784, acetaminophen 325 MG / pamabrom 25 MG Oral Tablet ctr 0 -18.322459970477404
602617, acetaminophen 325 MG / guaiFENesin 200 MG Oral Table

[nltk_data] Downloading package punkt to /home/jmandel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
def find_hits(q):
    qtokens = set([v.lower() for v in nltk.tokenize.word_tokenize(q)])
    print(qtokens)
    results = []
    for r in rxns:
        match = r['tokens'] & qtokens
        #print(r['tokens'] & qtokens, len(r['tokens'] & qtokens))
        if len(match) > 0:
            results.append([sum([10000/token_counts[t] for t in match]), r])
    results.sort(key=lambda v: v[0], reverse=True)
    for r in results[:100]:
        print(r)

find_hits("fluoxetine")
len(rxns)
#from txtai.embeddings import Embeddings

#embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2", "content": True})

#embeddings.index([(r["code"], r, None) for r in rxns])
#embeddings.save("rxnorm_prescribable")


{'fluoxetine'}
[625.0, {'code': '1190110', 'display': 'FLUoxetine HCl 60 MG Oral Tablet', 'text': 'FLUoxetine HCl 60 MG Oral Tablet', 'tty': 'SCD', 'tokens': {'hcl', 'mg', 'oral', 'fluoxetine', 'tablet', '60'}}]
[625.0, {'code': '248642', 'display': 'FLUoxetine 20 MG Oral Tablet', 'text': 'FLUoxetine 20 MG Oral Tablet', 'tty': 'SCD', 'tokens': {'20', 'mg', 'oral', 'fluoxetine', 'tablet'}}]
[625.0, {'code': '2532159', 'display': 'FLUoxetine 10 MG Oral Tablet (PMDD)', 'text': 'FLUoxetine 10 MG Oral Tablet (PMDD)', 'tty': 'SCD', 'tokens': {'10', 'mg', ')', 'pmdd', 'oral', 'fluoxetine', 'tablet', '('}}]
[625.0, {'code': '2532163', 'display': 'FLUoxetine 20 MG Oral Tablet (PMDD)', 'text': 'FLUoxetine 20 MG Oral Tablet (PMDD)', 'tty': 'SCD', 'tokens': {'20', 'mg', ')', 'pmdd', 'oral', 'fluoxetine', 'tablet', '('}}]
[625.0, {'code': '310384', 'display': 'FLUoxetine 10 MG Oral Capsule', 'text': 'FLUoxetine 10 MG Oral Capsule', 'tty': 'SCD', 'tokens': {'10', 'mg', 'capsule', 'oral', 'fluoxetine

29430

In [31]:
for r in embeddings.search("SELECT id, text, score FROM txtai WHERE similar('claritin 10 mg')  limit 100"):
    if r['score'] >= .45:
        print(r['id'], r['text'], r['score'])

199159 tretinoin 10 MG Oral Capsule 0.7951772212982178
485968 OLANZapine 10 MG Injection 0.7926617860794067
205535 PROzac 10 MG Oral Capsule 0.7905027270317078
199450 cloBAZam 10 MG Oral Tablet 0.7899085283279419
1114198 rivaroxaban 10 MG Oral Tablet 0.7846457958221436
313761 zaleplon 10 MG Oral Capsule 0.7836383581161499
211109 Clinidine 10 % Topical Solution 0.7825250625610352
1020023 ZyrTEC 10 MG Oral Capsule 0.7818443775177002
314154 OLANZapine 10 MG Oral Tablet 0.7809171676635742
352317 Strattera 10 MG Oral Capsule 0.779776930809021
484814 tadalafil 10 MG Oral Tablet 0.7795318365097046
991063 Bentyl 10 MG Oral Capsule 0.7790944576263428
2279434 Palforzia 10 MG Oral Powder 0.7774355411529541
904458 pravastatin sodium 10 MG Oral Tablet 0.7770956754684448
2272622 VALTOCO 10 MG Dose Kit 0.7759716510772705
1242611 MYORISAN 10 MG Oral Capsule 0.7754203081130981
835596 Tofranil 10 MG Oral Tablet 0.7740607857704163
1593858 Vyvanse 10 MG Oral Capsule 0.7738404273986816
208712 Cortef 10 MG 